In [1]:
from utils import *

In [2]:
from tqdm import tqdm

In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [4]:
np.random.seed(0)

In [5]:
time_range = np.arange(80, 112000, 20)

In [6]:
train_indeces = np.load('src/time_train_indeces.npy')
test_indeces = np.load('src/time_test_indeces.npy')

In [7]:
M = M_star = 5
N = NUMBER_OF_PEDESTRIANS
X = Y = 256

In [8]:
location = np.load('src/pedestrians_location.npy') # [NUMBER_OF_PEDESTRIANS x NUMBER_OF_FRAMES x 2]
location[:,:,0] = (X * location[:,:,0] / FRAME_WIDTH).astype(np.int64)
location[:,:,1] = (Y * location[:,:,1] / FRAME_HEIGHT).astype(np.int64)
# location[i,t] = [x,y] -- location of i-th pedestrian at time point t (aka frame 20*t) -- [0,0] for absent pedestians

ped_paths = np.load('src/all_not_ext_paths_with_ids.npy')
# ped_paths[j] = [i,t,x,y] -- all pedestrians' coordinates at all frames
ped_paths[:,2] = (X * ped_paths[:,2] / FRAME_WIDTH).astype(np.int64)
ped_paths[:,3] = (Y * ped_paths[:,3] / FRAME_HEIGHT).astype(np.int64)

In [9]:
# M-1 time points before curr_time and M_star after
def get_t(curr_time, M, M_star):
    step = 20
    return np.arange(curr_time - step*(M-1), curr_time + step*(M_star+1), step)

In [69]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
    
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=64, kernel_size=(2*M, 3, 3), bias=False)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(1, 3, 3), padding=(0, 1, 1), bias=False)
        self.conv3 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(1, 3, 3), padding=(0, 1, 1), bias=False)
        
        self.maxpool = nn.MaxPool3d(kernel_size=(1, 2, 2))
        
        # NEED TO ADD LOCATION BIAS MAP
        
        self.conv4 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(1, 3, 3), padding=(0, 1, 1), bias=False)
        self.conv5 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(1, 3, 3), padding=(0, 1, 1), bias=False)
        self.conv6 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(1, 3, 3), padding=(0, 1, 1), bias=False)
        
        self.deconv = nn.ConvTranspose3d(in_channels=64, out_channels=1, kernel_size=(2*M_star, 4, 4), stride=(1, 2, 2),
                                         bias=False)

    def forward(self, x):
        x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
        
#         x = self.maxpool(x)
        
        # NEED TO ADD LOCATION BIAS MAP
        
#         x = self.conv4(x)
#         x = self.conv5(x)
#         x = self.conv6(x)
        
#         x = self.deconv(x)
        
        return x
    

model = Net()
model = model.double()

In [67]:
batch_size = 5

dv = np.zeros((0, X, Y, 2*M))
dv_star = np.zeros((0, X, Y, 2*M_star))
m = np.zeros((0, X, Y, 2*M))

for curr_time in tqdm(time_range[train_indeces[:batch_size]]):
    t = get_t(curr_time, M, M_star)
    ind_t = (t/20).astype(int) # for indexing in location

    pedestrians_in_scene = ped_paths[ped_paths[:,1] == t[M-1]] # getting all pedestrians who located on map at curr_time

    disp_volume = np.zeros((X, Y, 2*M))
    disp_volume_star = np.zeros((X, Y, 2*M_star))

    for p in pedestrians_in_scene:
        l = location[p[0],ind_t[:M]] # locations of pedestrian at t_1, t_2, ..., t_M time points
        l_star = location[p[0],ind_t[M:]]

        # filling zeros absent pedestrians
        for i, row in enumerate(l):
            if (row == 0).all():
                l[i] = l[-1] + np.array([X, Y]) # to get zero in displacement vector

        for i, row in enumerate(l_star):
            if (row == 0).all():
                l_star[i] = l_star[-1] + np.array([X, Y])

        d = (l[-1] - l) / np.array([X, Y]) + 1
        d_star = (l_star[-1] - l_star) / np.array([X, Y]) + 1
        disp_volume[p[2], p[3]] = d.ravel() # put in [X, Y]-th cell of disp_volume displacement vector
        disp_volume_star[location[p[0],ind_t[-1]][0], location[p[0],ind_t[-1]][1]] = d_star.ravel()    

    mask = (disp_volume_star != 0).astype(int)
    
    dv = np.concatenate((dv, [disp_volume]))
    dv_star = np.concatenate((dv_star, [disp_volume_star]))
    m = np.concatenate((m, [mask]))


100%|██████████| 5/5 [00:00<00:00, 35.65it/s]

In [70]:
model(Variable(torch.from_numpy(dv)))

RuntimeError: 2D or 4D weight tensor expected, but got: [64 x 1 x 10 x 3 x 3] at /data/users/soumith/miniconda2/conda-bld/pytorch-cuda80-0.1.10_1488757768560/work/torch/lib/THNN/generic/SpatialConvolutionMM.c:15

In [ ]:
# THERE IS A PROBLEM HERE:
# we lose pedestrians' ids after coding into disp_volume so to decode we need somehow match previous path and predicted

# displacement volume decode
xs, ys = disp_volume[:,:,0].nonzero() # getting indices of non-zero predicted coordinates at first (t_M+1) timepoint
for i, j in zip(xs,ys):
    np.array([i,j]) - (disp_volume[i,j].reshape(-1,2) - 1) * np.array([X, Y])